In [1]:
from typing import List

from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.retrievers import BaseRetriever, Document
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.trace.export import ConsoleSpanExporter, SimpleSpanProcessor

from openinference.instrumentation.langchain import LangChainInstrumentor


In [ ]:
import os
import phoenix as px

os.environ["PHOENIX_TRACING"] = "true"
px.launch_app()

In [2]:

endpoint = "http://127.0.0.1:6006/v1/traces"
tracer_provider = trace_sdk.TracerProvider()
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))
tracer_provider.add_span_processor(SimpleSpanProcessor(ConsoleSpanExporter()))

LangChainInstrumentor().instrument(tracer_provider=tracer_provider)


In [3]:


class CustomRetriever(BaseRetriever):
    """
    This example is taken from langchain docs.

    https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/custom_retriever/

    A custom retriever that contains the top k documents that contain the user query.

    This retriever only implements the sync method _get_relevant_documents.

    If the retriever were to involve file access or network access, it could benefit
    from a native async implementation of `_aget_relevant_documents`.

    As usual, with Runnables, there's a default async implementation that's provided
    that delegates to the sync implementation running on another thread.
    """

    k: int
    """Number of top results to return"""

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        """Sync implementations for retriever."""
        matching_documents: List[Document] = []

        # Custom logic to find the top k documents that contain the query

        for index in range(self.k):
            matching_documents.append(Document(page_content=f"dummy content at {index}", score=1.0))
        return matching_documents



In [4]:

retriever = CustomRetriever(k=3)


if __name__ == "__main__":
    documents = retriever.invoke("what is the meaning of life?")

{
    "name": "CustomRetriever",
    "context": {
        "trace_id": "0xda9bcad28a6ef6c1a930dc2f3c992e41",
        "span_id": "0xdceb01fd75aaab0b",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": null,
    "start_time": "2025-02-06T06:34:26.789712Z",
    "end_time": "2025-02-06T06:34:26.790748Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {
        "input.value": "what is the meaning of life?",
        "output.value": "{\"documents\": [\"page_content='dummy content at 0'\", \"page_content='dummy content at 1'\", \"page_content='dummy content at 2'\"]}",
        "output.mime_type": "application/json",
        "retrieval.documents.0.document.content": "dummy content at 0",
        "retrieval.documents.1.document.content": "dummy content at 1",
        "retrieval.documents.2.document.content": "dummy content at 2",
        "metadata": "{\"ls_retriever_name\": \"custom\"}",
        "openinference.span.kind": "RETRIEVER"
    },